<a href="https://colab.research.google.com/github/Metallicode/Math/blob/main/Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train -- Validation  --  Test

In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [125]:
df = pd.read_csv("Advertising.csv")

X = df.drop(["sales"], axis=1)
y = df["sales"]

In [45]:
df[df.isnull()].count()

TV           0
radio        0
newspaper    0
sales        0
dtype: int64

In [ ]:
X

In [ ]:
y

In [3]:
len(df)

200

In [46]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3, shuffle=True)

In [47]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()

scalar.fit(X_train)

X_train = scalar.transform(X_train)
X_ = scalar.transform(X_)

In [48]:
X_validate, X_test, y_validate, y_test = train_test_split(X_, y_, test_size=0.5, shuffle=True)

In [49]:
len(X_train), len(X_validate), len(X_test)

(140, 30, 30)

##Create and Compare model evaluations


In [50]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [74]:
best_model = None
ridge_alpha = 10
best_RMSE = np.inf
last_model = None

while best_model is None:

  model = Ridge(alpha=ridge_alpha)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_validate)
  RMSE = np.sqrt(mean_squared_error(y_pred, y_validate))

  if best_RMSE > RMSE and ridge_alpha > 0.01:
    best_RMSE = RMSE

  else:
    best_model=last_model
    break

  last_model = model
  ridge_alpha = ridge_alpha-0.01

In [75]:
best_RMSE

1.9357255440568473

In [76]:
ridge_alpha

1.6888920817414999e-13

In [77]:
y_pred = best_model.predict(X_test)
RMSE = np.sqrt(mean_squared_error(y_pred, y_test))

In [78]:
RMSE

1.4587894987840422

#Cross Validation

In [126]:
from sklearn.model_selection import cross_val_score

###Scoring

[model_evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html)



In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)
scalar = StandardScaler()
scalar.fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

best_model = None
ridge_alpha = 10
best_score = np.inf
last_model = None


while best_model is None:


  model = Ridge(alpha=ridge_alpha)

  cv_score = cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=5)

  score = np.sqrt(abs(cv_score.mean()))

  if best_score > score and ridge_alpha > 0.01:
    best_score = score

  else:
    best_model=last_model
    break

  last_model = model
  ridge_alpha = ridge_alpha-0.01

In [128]:
ridge_alpha

0.35000000000016906

In [129]:
Final_Model = Ridge(alpha=ridge_alpha)

Final_Model.fit(X_train, y_train)

y_pred = Final_Model.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_pred, y_test))

In [130]:
RMSE


1.6934766013408833

##sklearn.model_selection **cross_validate**

In [131]:
from sklearn.model_selection import cross_validate

In [132]:
Final_Model = Ridge(alpha=ridge_alpha)

#SCORING strings
#https://scikit-learn.org/stable/modules/model_evaluation.html
scoring = cross_validate(Final_Model, X_train, y_train, cv=5, scoring=["neg_mean_squared_error",
                                                                        "neg_mean_absolute_error"] )


In [133]:
scoring_df = pd.DataFrame(scoring)

In [135]:
scoring_df.mean()


fit_time                        0.002748
score_time                      0.001473
test_neg_mean_squared_error    -3.088105
test_neg_mean_absolute_error   -1.317669
dtype: float64